In [1]:
import glob
import hashlib
import os

import numpy as np
import pandas as pd
import tensorflow as tf
keras = tf.keras

/home/silent/projects/kaggle-tensorflow-speech-recognition-challenge/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
LABELS = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown']
NUM_FOLDS = 5

In [3]:
metadata = {}

In [4]:
def collect_files(directory):
    metadata = {}
    rel_start = len(directory)
    for path in glob.glob(directory + '/**/*.wav'):
        rel_path = path[rel_start + 1:]
        label, fname = os.path.split(path)
        label = os.path.basename(label)
        speaker = fname.split('_')[0]
        try:
            metadata[label][speaker].append(path)
        except KeyError:
            try:
                metadata[label][speaker] = [path]
            except KeyError:
                metadata[label] = {speaker: [path]}    
        
    return metadata

In [5]:
metadata = collect_files('../data/train/audio')

In [6]:
metadata_summary = pd.DataFrame([
    {'label': label,
     'speaker': speaker,
     'filename': filename}
    for label, v in metadata.items()
    for speaker, files in v.items()
    for filename in files
]).groupby('label').agg({'speaker': lambda x: len(np.unique(x)), 'filename': 'count'})
metadata_summary

,speaker,filename
label,,
_background_noise_,6,6
bed,1177,1713
bird,1194,1731
cat,1180,1733
dog,1209,1746
down,1206,2359
eight,1179,2352
five,1181,2357
four,1194,2372


In [7]:
splits = [[] for _ in range(NUM_FOLDS)]
for label, v in metadata.items():
    if label == '_background_noise_':
        continue
    for speaker, files in v.items():
        speaker_hash = hashlib.sha1(speaker.encode('utf-8')).hexdigest()
        split_num = int(speaker_hash, 16) % NUM_FOLDS
        splits[split_num].extend([(fname, label) for fname in files])

In [8]:
[len(s) for s in splits]

[12648, 13396, 12766, 11950, 13961]

## Determine number of silence samples to generate

In [9]:
unknown_summary = metadata_summary.iloc[~metadata_summary.index.isin(LABELS)]
unknown_summary.sum()

speaker     23699
filename    41045
dtype: int64

Could either match the number of silence samples with the unknown class or downsample both silence and unknown classes to be approximately balanced with other classes. 